In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
import pandas as pd
from rdkit import Chem
from ast import literal_eval

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def inchikey(array): #returns first layer of InChIKey (excluding stereochemical information), passed array of dataframe containing the SMILES for a molecule as its 2nd element
    Smiles = array[1]
    mol = Chem.MolFromSmiles(Smiles)
    dummy = (Chem.inchi.MolToInchiKey(mol))[:14] #retrieves first layer only
    return(dummy)

In [ ]:
def find_matches(library_set, test_set): #creates dataframe containing 'matched' molecules common between the products of a CRN and the library of nucleoside analogues, passed the filepaths for dataframes containing these 2 datasets
  library_data = pd.read_csv(library_set, sep='\t')
  test_data = pd.read_csv(test_set, sep='\t')
  test_data['INCHIKEY'] = test_data.apply(inchikey, axis=1, raw=True, result_type='expand') #evaluates InChIKey of all CRN output molecules

  library_codes = list(library_data['INCHIKEY'])
  test_codes = list(test_data['INCHIKEY'])
  matches_data = [[], [], [], []]
  for i in range(len(test_codes)):
    if i%1000 == 0:
      print(i)
    if test_codes[i] in library_codes:
      library_index = library_codes.index(test_codes[i])
      matches_data[0].append(test_data['Generation'][i])
      matches_data[1].append(test_data['INCHIKEY'][i])
      matches_data[2].append(test_data['Smiles'][i])
      matches_data[3].append(literal_eval(library_data['SMILES'][library_index])) #literal_eval required to interpret content of SMILES column in the library dataset which is stored as strings
  df = pd.DataFrame({'Generation':matches_data[0], 'INCHIKEY':matches_data[1], 'NetworkSmiles':matches_data[2], 'AnalogueSmiles':matches_data[3]})
  df.to_csv('/content/drive/MyDrive/BMSIS /MinimalDirectory/ProcessedData/HCNMatches.tsv', sep='\t')
  return(df)

In [ ]:
# %%time
# a = find_matches('/content/drive/MyDrive/BMSIS /MinimalDirectory/ProcessedData/Nucleoside_Stereoisomers.tsv', '/content/drive/MyDrive/BMSIS /MinimalDirectory/OriginalData/OriginalNetworkData/Products/hcn_output.tsv')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
CPU times: user 38.6 s, sys: 277 ms, total: 38.9 s
Wall time: 39.6 s
